In [64]:
import qiskit
from qiskit import QuantumCircuit, BasicAer, Aer, assemble, transpile
from qiskit.quantum_info import random_statevector
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.opflow import I, X, Z
from qiskit.algorithms.optimizers import ADAM
from qiskit.circuit import ParameterVector
from qiskit.algorithms import VQE

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

from random import random
import numpy as np
from statistics import mean
from scipy.optimize import minimize
from tqdm import tqdm
import itertools

In [65]:
def add_layers(circuit, params):
    
    depth = len(params) // 2
    num_qubits = circuit.num_qubits

    for p in range(0, len(params), 2):
        
        beta = params[p]
        gamma = params[p + 1]
        
        i = 0
        while i < num_qubits - 1:
            circuit.rzz(beta, i, i+1)
            i = i + 2

        if num_qubits > 2:
            circuit.rzz(beta, 0, num_qubits-1)

        j = 1
        while j < num_qubits - 1:
            circuit.rzz(beta, j, j+1)        
            j = j + 2

        circuit.barrier()

        for k in range(num_qubits):
            circuit.rx(gamma, k)

        circuit.barrier()

    return circuit

In [66]:
def create_random_param(depth):
    
    a = random() * np.pi
    b = random() * np.pi
    params = []
    for _ in range(depth):
        params.append(a)
        params.append(b)
        
    return params
    
#     params = []
#     for _ in range(depth * 2):
#         params.append(random() * np.pi)
        
    return params

In [67]:
def create_state_vector(num_qubits, params):
    
    circuit = QuantumCircuit(num_qubits)
    
    for i in range(num_qubits):
        circuit.h(i)
    circuit.barrier()
    
    add_layers(circuit, params)
        
    svsim = Aer.get_backend('statevector_simulator')
    qobj = assemble(circuit)
    state_vector = svsim.run(qobj).result().get_statevector()

    return state_vector

In [68]:
#Made by Anirban
def Haar(num_qubits):
    #defining a random n qubit Haar random state
    state=random_statevector(2**num_qubits).data
    amplitudes = state
    bit_arr = [list(i) for i in itertools.product([0, 1], repeat = num_qubits)]
    bitStr_arr=[''.join([str(bit_arr[i][j]) for j in range(len(bit_arr[i]))]) for i in range(len(bit_arr))]
    #String rep. for the random state
    Psi=[bitStr_arr[i] + ',' + str(amplitudes[i]) for i in range(len(amplitudes)) if np.abs(amplitudes[i]) > 1e-5]
    return Psi, state

In [69]:
def schmidt_spectrum(state):
    matrix = np.array(state).reshape(int(np.sqrt(len(state))), int(np.sqrt(len(state))))
    u,vector,v = np.linalg.svd(matrix)
    return vector

In [79]:
#"""""""""""""""""""""" Graph config """""""""""""""""""#
colors = plt.cm.viridis(np.linspace(0,1,17))
fig, ax = plt.subplots(dpi=100)
ax.set_xlabel(r'$k$')
ax.set_ylabel(r'$\xi_k$')
norm = mpl.colors.BoundaryNorm(list(range(17)), plt.cm.viridis.N)
scalar_mappable = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis)
clb = fig.colorbar(scalar_mappable)
clb.ax.set_title(r'$p$')
#""""""""""""""""""""""""""""""""""""""""""""""""""""""#

num_qubits = 16
iterations = 5
depth_range = range(1,num_qubits + 1)

for depth in tqdm(depth_range):
    all_es = []
    for _ in range(iterations):
        random_param = create_random_param(depth)
        state = create_state_vector(num_qubits, random_param)
        schmidt_coeffs = schmidt_spectrum(state)
        all_es.append(np.log(schmidt_coeffs))
    mean_es = np.array(all_es, dtype=object).mean(axis=0)
    ax.plot(mean_es, linestyle='solid', markersize=2, color=colors[depth])

all_haar = []
for i in tqdm(range(iterations)):
    Psi,state=Haar(num_qubits)
    schmidt_coefficients=schmidt_spectrum(state)
    spectrum=np.log(schmidt_coefficients)
    all_haar.append(spectrum)
mean_haar = np.array(all_haar, dtype=object).mean(axis=0)
ax.plot(mean_haar, linestyle='dotted', color='#000000', markersize=2)
    
plt.show()

In [80]:
seed = algorithm_globals.random_seed
quant_inst = QuantumInstance(BasicAer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

repetitions = 1
num_qubits = 4
depth_range = range(2,num_qubits**2 + 1, 2)
hamiltonian = (Z^I^I^Z) + (Z^Z^I^I) + (I^Z^Z^I) + (I^I^Z^Z) + (X^I^I^I) + (I^X^I^I) + (I^I^X^I) + (I^I^I^X)
optimizer = ADAM(maxiter=3000,tol=0.0001, lr=0.001)

evals_list = []
for depth in tqdm(depth_range):
    total_evals = []
    for repetition in range(repetitions):
        initial = create_random_param(depth)
        ansatz = create_circuit(num_qubits, ParameterVector('p',depth*2))
        vqe = VQE(ansatz = ansatz, optimizer = optimizer, initial_point = initial, quantum_instance=quant_inst)
        result = vqe.compute_minimum_eigenvalue(operator = hamiltonian)
        evals = result.cost_function_evals
        total_evals.append(evals)
    mean_evals = np.array(total_evals, dtype=object).mean(axis=0)
    evals_list.append(mean_evals)

In [ ]:
plt.plot(depth_range, evals_list, linestyle='solid', markersize=2)
plt.xlabel('depth')
plt.ylabel('iterations')
plt.show()

In [ ]:
def expectation_value(counts, basis, shots):
    
    def eigenvalue_mapping(bit):
        return 1 if int(bit) == 1 else -1
        
    energies = []

    if basis == 'z':
        #Go through every repetition
        for key, value in counts.items():
            energy = 0     
            #boundary condition
            leftmost = key[0]
            rightmost = key[len(key) - 1]
            energy += eigenvalue_mapping(leftmost) * eigenvalue_mapping(rightmost)
            
            #go through each bitstring, bit-by-bit
            for j in range(len(key) - 1):
                left = key[j]
                right = key[j+1]
            
                #add each count of this bitstring
                for k in range(value):
                    energy += eigenvalue_mapping(left) * eigenvalue_mapping(right)
            energies.append(energy)
    
    else:
        for key, value in counts.items():
            energy = 0
            for j in range(len(key)):
                bit = key[j]
                
                for k in range(value):
                    energy += eigenvalue_mapping(bit)
                    
            energies.append(energy)
    
    return np.sum(energies) / shots

In [ ]:
def cost_function(params):

    num_qubits = 4
    g_field = 1
    shots = 5000
    simulator = Aer.get_backend('aer_simulator')

    # Z basis measurement
    circuit = create_circuit(num_qubits, params)  
    circuit.measure_all()
    circuit = transpile(circuit, simulator)
    result = simulator.run(circuit, shots = shots).result()
    counts = result.get_counts(circuit)
    z_exp_value = expectation_value(counts, 'z', shots)

    # X basis measurement
    circuit = create_circuit(num_qubits, params)
    for i in range(num_qubits):
        circuit.h(i)
    circuit.measure_all()
    circuit = transpile(circuit, simulator)
    result = simulator.run(circuit, shots = shots).result()
    counts = result.get_counts(circuit)
    x_exp_value = expectation_value(counts, 'x', shots)
    
    exp_value = z_exp_value + g_field * x_exp_value
        
    return exp_value